In [1]:
import cv2
import os
import base64
import requests
import pickle
import numpy as np

In [2]:
url = "http://localhost:8080/api/gethog"

classB = ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce','Swift', 'Tata Safari', 'Toyota Innova']

def img2HOG(img):
    v, buffer = cv2.imencode(".jpg", img)
    img_str = base64.b64encode(buffer)
    data = "image data,"+str.split(str(img_str),"'")[1]
    response = requests.get(url, json={"img":data})
    
    return response.json()

In [3]:
def readData(path):
    response = []
    for sub in os.listdir(path):
        for fn in os.listdir(path + '/' + sub):
            img_file_name = path + '/' + sub + "/" + fn
            img = cv2.imread(img_file_name)
            res = img2HOG(img)
            hog = list(res["hog"])
            hog.append(classB.index(sub))
            response.append(hog)
    return response

In [4]:
def savePkl(filename ,path):
    cars = readData(path)
        
    write_path = filename + ".pkl"
    pickle.dump(cars, open(write_path,"wb"))
    print("data preparation is done")
 
def loadPkl(filename):
    dataset = pickle.load(open(filename + '.pkl','rb'))
    return dataset

In [5]:
train_dir = r'Cars Dataset\train'
savePkl('train_cars',train_dir)

data preparation is done


In [6]:
test_dir = r'Cars Dataset\test'
savePkl('test_cars',test_dir)

data preparation is done


In [7]:
dataset_train = loadPkl('train_cars')
print("Data train : ",len(dataset_train))
dataset_test = loadPkl('test_cars')
print("Data test : ",len(dataset_test))

Data train :  3352
Data test :  813


In [8]:
train_arr = np.array(dataset_train)
x_train = train_arr[:,0:-1]
y_train = train_arr[:,-1]

In [9]:
test_arr = np.array(dataset_test)
x_test = test_arr[:,0:-1]
y_test = test_arr[:,-1]

In [10]:
pip install scikit-learn==1.2.2



  Using cached scikit_learn-1.2.2-cp311-cp311-win_amd64.whl (8.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0


In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


In [12]:
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [13]:
y_pred = clf.predict(x_test)
print("Accuracy:",accuracy_score(y_test, y_pred)*100)
print("Confusion Matrix : \n",confusion_matrix(y_test,y_pred))

Accuracy: 47.35547355473555
Confusion Matrix : 
 [[ 76  17  18  17  17  24  30]
 [  7  41   4   4   1   2   8]
 [ 11   5  24   6   5  10  14]
 [ 28   3   7  10   7   7  12]
 [ 15  11   5   4  45   2  20]
 [ 19   4   2   5   2  65   9]
 [ 23   7   8  11  10   7 124]]


In [14]:
path_model ='Car_Brand_Model.pkl'
pickle.dump(clf, open(path_model,'wb'))